In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install alias-free-torch
!pip install einops-exts
!pip install k_diffusion einops torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# ***Generate music using the original model***

In [2]:
import torch
import torchaudio
from einops import rearrange
import sys
sys.path.append('/content/drive/MyDrive/kings project/stable-audio-tools-main')

from stable_audio_tools import get_pretrained_model
from stable_audio_tools.inference.generation import generate_diffusion_cond

device = "cuda" if torch.cuda.is_available() else "cpu"

# Download model
model, model_config = get_pretrained_model("stabilityai/stable-audio-open-small")
sample_rate = model_config["sample_rate"]
sample_size = model_config["sample_size"]

model = model.to(device)

# Set up text and timing conditioning
conditioning = [{
    "prompt": "A traditional Chinese folk tune played with the guzheng and dizi, slow and peaceful.",
    "seconds_total": 11
}]

# Generate stereo audio
output = generate_diffusion_cond(
    model,
    steps=8,
    cfg_scale=1.0,
    conditioning=conditioning,
    sample_size=sample_size,
    sampler_type="pingpong",
    device=device
)

# Rearrange audio batch to a single sequence
output = rearrange(output, "b d n -> d (b n)")

# Peak normalize, clip, convert to int16, and save to file
output = output.to(torch.float32).div(torch.max(torch.abs(output))).clamp(-1, 1).mul(32767).to(torch.int16).cpu()
torchaudio.save("Prompt1_1.wav", output, sample_rate)

No module named 'flash_attn'
flash_attn not installed, disabling Flash Attention


/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

969807566


/content/drive/MyDrive/kings project/stable-audio-tools-main/stable_audio_tools/models/conditioners.py:362: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.float16) and torch.set_grad_enabled(self.enable_grad):
100%|██████████| 8/8 [00:00<00:00, 17.89it/s]


In [37]:
import torch
import torchaudio
from einops import rearrange
import sys
sys.path.append('/content/drive/MyDrive/kings project/stable-audio-tools-main')

from stable_audio_tools import get_pretrained_model
from stable_audio_tools.inference.generation import generate_diffusion_cond

device = "cuda" if torch.cuda.is_available() else "cpu"

# Download model
model, model_config = get_pretrained_model("stabilityai/stable-audio-open-small")
sample_rate = model_config["sample_rate"]
sample_size = model_config["sample_size"]

model = model.to(device)

prompts = [
    "A traditional Chinese folk tune played with the guzheng and dizi, slow and peaceful.",
    "Upbeat Chinese festival music with gongs, drums, and joyful sheng melody.",
    "A sorrowful Chinese ballad with erhu and soft piano background.",
    "Ancient Chinese imperial court music with pipa and temple bells.",
    "Modern Chinese pop fusion with R&B elements and traditional instruments like guzheng."
]

for i, prompt in enumerate(prompts, 1):
    conditioning = [{
        "prompt": prompt,
        "seconds_total": 11
    }]

    output = generate_diffusion_cond(
        model,
        steps=8,
        cfg_scale=1.0,
        conditioning=conditioning,
        sample_size=sample_size,
        sampler_type="pingpong",
        device=device
    )

    output = rearrange(output, "b d n -> d (b n)")
    output = output.to(torch.float32).div(torch.max(torch.abs(output))).clamp(-1, 1).mul(32767).to(torch.int16).cpu()

    filename = f"Prompt{i}.wav"
    torchaudio.save(filename, output, sample_rate)
    print(f"Saved {filename}")

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/content/drive/MyDrive/kings project/stable-audio-tools-main/stable_audio_tools/models/conditioners.py:362: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.float16) and torch.set_grad_enabled(self.enable_grad):


3631400547


100%|██████████| 8/8 [00:00<00:00, 29.52it/s]


Saved Prompt1.wav
3271351814


100%|██████████| 8/8 [00:00<00:00, 29.65it/s]


...

Saved Prompt3.wav
1437641699


100%|██████████| 8/8 [00:00<00:00, 29.69it/s]


Saved Prompt4.wav
36405571


100%|██████████| 8/8 [00:00<00:00, 29.76it/s]


Saved Prompt5.wav


# **Reconstructing audio after fine-tuning**

In [39]:
import sys
import os
import json
import torch
import torchaudio

root_path = "/content/drive/MyDrive/kings project/stable-audio-tools-main"
if root_path not in sys.path:
    sys.path.append(root_path)

from stable_audio_tools.models import autoencoders


model_config_path = "/content/drive/MyDrive/kings project/Finetuned Model/model_config.json"
ckpt_path = "/content/drive/MyDrive/kings project/Finetuned Model/epoch=12-step=100.ckpt"
input_audio_path = "/content/Prompt2.wav" # Manually modify the input audio here
output_audio_path = "/content/Prompt2_reconstructed.wav"

# Read model configuration
with open(model_config_path, 'r') as f:
    model_config = json.load(f)

# Creating Models
model = autoencoders.create_autoencoder_from_config(model_config)

# load weight
checkpoint = torch.load(ckpt_path, map_location='cpu')
state_dict = checkpoint.get('state_dict', checkpoint)

# Processing weights key prefix
new_state_dict = {}
for k, v in state_dict.items():
    new_key = k
    if k.startswith('model.'):
        new_key = k[len('model.'):]
    new_state_dict[new_key] = v

model.load_state_dict(new_state_dict, strict=False)
model.eval()

# Read Input Audio
waveform, sample_rate = torchaudio.load(input_audio_path)

# Resample if sampling rate does not match
target_sr = model.sample_rate
if sample_rate != target_sr:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sr)
    waveform = resampler(waveform)

waveform = waveform.unsqueeze(0)  # Add batch dimension

# Encoding-Decoding Process
with torch.no_grad():
    latents = model.encode_audio(waveform)
    reconstructed = model.decode_audio(latents)

# Removing the batch dimension
reconstructed = reconstructed.squeeze(0)

# Save Reconstructed Audio
torchaudio.save(output_audio_path, reconstructed, target_sr)

print(f"The reconstructed audio has been saved to：{output_audio_path}")

The reconstructed audio has been saved to：/content/Prompt2_reconstructed.wav


In [42]:
# call epoch=99-step=800.ckpt

model_config_path = "/content/drive/MyDrive/kings project/Finetuned Model/model_config.json"
ckpt_path = "/content/drive/MyDrive/kings project/Finetuned Model/epoch=99-step=800.ckpt"
input_audio_path = "/content/Prompt2.wav" # Manually modify the input audio here
output_audio_path = "/content/Prompt2_reconstructed.wav"

# Read model configuration
with open(model_config_path, 'r') as f:
    model_config = json.load(f)

# Creating Models
model = autoencoders.create_autoencoder_from_config(model_config)

# load weight
checkpoint = torch.load(ckpt_path, map_location='cpu')
state_dict = checkpoint.get('state_dict', checkpoint)

# Processing weights key prefix
new_state_dict = {}
for k, v in state_dict.items():
    new_key = k
    if k.startswith('model.'):
        new_key = k[len('model.'):]
    new_state_dict[new_key] = v

model.load_state_dict(new_state_dict, strict=False)
model.eval()

# Read Input Audio
waveform, sample_rate = torchaudio.load(input_audio_path)

# Resample if sampling rate does not match
target_sr = model.sample_rate
if sample_rate != target_sr:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sr)
    waveform = resampler(waveform)

waveform = waveform.unsqueeze(0)  # Add batch dimension

# Encoding-Decoding Process
with torch.no_grad():
    latents = model.encode_audio(waveform)
    reconstructed = model.decode_audio(latents)

# Removing the batch dimension
reconstructed = reconstructed.squeeze(0)

# Save Reconstructed Audio
torchaudio.save(output_audio_path, reconstructed, target_sr)

print(f"The reconstructed audio has been saved to：{output_audio_path}")

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [34]:
# Read Input Audio
waveform, sample_rate = torchaudio.load(input_audio_path)
print("Input waveform shape:", waveform.shape, "Sample rate:", sample_rate)

# Make sure it's Dual or Mono, convert to Dual (2) example:
if waveform.shape[0] == 1:
    waveform = waveform.repeat(2, 1)

# resample
target_sr = model.sample_rate
if sample_rate != target_sr:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sr)
    waveform = resampler(waveform)

waveform = waveform.unsqueeze(0)  # batch dimension

# Normalization - depending on model preprocessing requirements
# waveform = waveform / waveform.abs().max()

with torch.no_grad():
    # Use chunked=True for long audio with an adjustable parameter
    latents = model.encode_audio(waveform, chunked=True, overlap=64, chunk_size=256)
    reconstructed = model.decode_audio(latents, chunked=True, overlap=64, chunk_size=256)

# Go to batch dimension
reconstructed = reconstructed.squeeze(0)

# output check
print("Reconstructed shape:", reconstructed.shape)
print("Max amplitude:", reconstructed.abs().max().item())

# Save Audio
torchaudio.save(output_audio_path, reconstructed, target_sr)
print(f"The reconstructed audio has been saved to：{output_audio_path}")

Input waveform shape: torch.Size([2, 524288]) Sample rate: 44100
Reconstructed shape: torch.Size([2, 524288])
Max amplitude: 0.2103666514158249
The reconstructed audio has been saved to：/content/Prompt2_reconstructed.wav


In [32]:
new_state_dict = {}
for k, v in state_dict.items():
    new_key = k
    if new_key.startswith('model.'):
        new_key = new_key[len('model.'):]
    if new_key.startswith('autoencoder.'):
        new_key = new_key[len('autoencoder.'):]
    new_state_dict[new_key] = v

In [30]:
missing_keys, unexpected_keys = model.load_state_dict(new_state_dict, strict=False)
print("Missing keys:", missing_keys)
print("Unexpected keys:", unexpected_keys)

Missing keys: []
Unexpected keys: ['sdstft.mrstft.stft_losses.0.prefilter.fir.weight', 'sdstft.mrstft.stft_losses.1.prefilter.fir.weight', 'sdstft.mrstft.stft_losses.2.prefilter.fir.weight', 'sdstft.mrstft.stft_losses.3.prefilter.fir.weight', 'sdstft.mrstft.stft_losses.4.prefilter.fir.weight', 'sdstft.mrstft.stft_losses.5.prefilter.fir.weight', 'sdstft.mrstft.stft_losses.6.prefilter.fir.weight', 'lrstft.stft_losses.0.prefilter.fir.weight', 'lrstft.stft_losses.1.prefilter.fir.weight', 'lrstft.stft_losses.2.prefilter.fir.weight', 'lrstft.stft_losses.3.prefilter.fir.weight', 'lrstft.stft_losses.4.prefilter.fir.weight', 'lrstft.stft_losses.5.prefilter.fir.weight', 'lrstft.stft_losses.6.prefilter.fir.weight', 'discriminator.discriminators.discriminators.0.spec_transform.window', 'discriminator.discriminators.discriminators.0.convs.0.conv.bias', 'discriminator.discriminators.discriminators.0.convs.0.conv.weight_g', 'discriminator.discriminators.discriminators.0.convs.0.conv.weight_v', 'discr